In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_excel("raw_chennai.xlsx")

In [12]:
df

,Unnamed: 0,name,location,price,area,bhk
0,0,on request,"2 BHK Flat in Pallikaranai, Chennai",₹48 Lac,764 sqft,2 BHK
1,1,RC Adena,"2 BHK Flat in Thirumullaivoyal, Chennai",₹45 Lac,820 sqft,2 BHK
2,2,Radiance Suprema,"2 BHK Flat in Madhavaram, Chennai",₹1 Cr,"1,102 sqft",2 BHK
3,3,on request,"2 BHK Flat in Pallikaranai, Chennai",₹57.7 Lac,932 sqft,2 BHK
4,4,Shriram Park 63\n3.8,"3 BHK Flat in Perungalathur, Chennai",₹1.5 Cr,"1,970 sqft",3 BHK
...,...,...,...,...,...,...
291,291,TDC VATIKA,"3 BHK Flat in Bhuvaneshwari Nagar, Velachery",₹69 Lac,990 sqft,3 BHK
292,292,Malles Ahaana,"2 BHK Flat in Pallikaranai, Chennai",₹65 Lac,989 sqft,2 BHK
293,293,lake vista's,"3 BHK Flat in Pallikaranai, Chennai",₹1.2 Cr,"2,139 sqft",3 BHK
294,294,"Moulivakkam, Chennai, Chennai West","3 Bedroom House in Moulivakkam, Chennai",₹1.4 Cr,900 sqft,3 BHK


In [6]:
df = df.drop(columns=[col for col in df.columns if "Unnamed" in col])

In [7]:
df

,name,location,price,area,bhk
0,on request,"2 BHK Flat in Pallikaranai, Chennai",₹48 Lac,764 sqft,2 BHK
1,RC Adena,"2 BHK Flat in Thirumullaivoyal, Chennai",₹45 Lac,820 sqft,2 BHK
2,Radiance Suprema,"2 BHK Flat in Madhavaram, Chennai",₹1 Cr,"1,102 sqft",2 BHK
3,on request,"2 BHK Flat in Pallikaranai, Chennai",₹57.7 Lac,932 sqft,2 BHK
4,Shriram Park 63\n3.8,"3 BHK Flat in Perungalathur, Chennai",₹1.5 Cr,"1,970 sqft",3 BHK
...,...,...,...,...,...
291,TDC VATIKA,"3 BHK Flat in Bhuvaneshwari Nagar, Velachery",₹69 Lac,990 sqft,3 BHK
292,Malles Ahaana,"2 BHK Flat in Pallikaranai, Chennai",₹65 Lac,989 sqft,2 BHK
293,lake vista's,"3 BHK Flat in Pallikaranai, Chennai",₹1.2 Cr,"2,139 sqft",3 BHK
294,"Moulivakkam, Chennai, Chennai West","3 Bedroom House in Moulivakkam, Chennai",₹1.4 Cr,900 sqft,3 BHK


In [8]:
df_cleaned = (
    df
    .drop_duplicates()
    .apply(lambda col: col.str.strip().str.lower() if col.dtype == "object" else col)
    .assign(
        is_starred=lambda df_: df_.name.str.contains("\n").astype(int),
        name=lambda df_: (
            df_
            .name
            .str.replace("\n[0-9.]+", "", regex=True)
            .str.strip()
            .replace("adroit district s", "adroit district's")
        ),
        location=lambda df_: (
            df_
            .location
            .str.replace("chennai", "")
            .str.strip()
            .str.replace(",$", "", regex=True)
            .str.split("in")
            .str[-1]
            .str.strip()
        ),
        price=lambda df_: (
            df_
            .price
            .str.replace("₹", "")
            .apply(lambda val: float(val.replace("lac", "").strip()) if "lac" in val else float(val.replace("cr", "").strip()) * 100)
        ),
        area=lambda df_: (
            df_
            .area
            .str.replace("sqft", "")
            .str.strip()
            .str.replace(",", "")
            .pipe(pd.to_numeric)
        ),
        bhk=lambda df_: (
            df_
            .bhk
            .str.replace("bhk", "")
            .str.strip()
            .pipe(pd.to_numeric)
        )
    )
    .rename(columns={
        "price": "price_lakhs",
        "area": "area_sqft"
    })
    .reset_index(drop=True)
)


In [9]:
df_cleaned

,name,location,price_lakhs,area_sqft,bhk,is_starred
0,on request,pallikaranai,48.0,764,2,0
1,rc adena,thirumullaivoyal,45.0,820,2,0
2,radiance suprema,madhavaram,100.0,1102,2,0
3,on request,pallikaranai,57.7,932,2,0
4,shriram park 63,perungalathur,150.0,1970,3,1
...,...,...,...,...,...,...
278,tdc vatika,"bhuvaneshwari nagar, velachery",69.0,990,3,0
279,malles ahaana,pallikaranai,65.0,989,2,0
280,lake vista's,pallikaranai,120.0,2139,3,0
281,"moulivakkam, chennai, chennai west",moulivakkam,140.0,900,3,0


In [10]:
df_cleaned.to_excel("chennai-properties-cleaned.xlsx", index=False)
